In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import shutil
from PIL import Image  

In [2]:
def get_mean_and_std(loader):
    mean = 0
    std = 0
    total_image_count = 0
    for images, _ in loader:
        image_count_in_a_batch = images.size(0)
        images = images.view(image_count_in_a_batch, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_image_count += image_count_in_a_batch
    mean /= total_image_count
    std /= total_image_count
    return mean, std

In [3]:
def set_device():
    return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    device = set_device()
    model.to(device)
    best_acc = 0
    
    for epoch in range(n_epochs):
        print(f"Epoch {epoch+1}")
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * running_correct / total
        print(f" - Training Accuracy: {epoch_acc:.2f}%. Loss: {epoch_loss:.4f}")
        
        if test_loader:
            test_acc = evaluate_model_on_test_set(model, test_loader)
            if test_acc > best_acc:
                best_acc = test_acc
                save_checkpoint(model, epoch, optimizer, best_acc)
    
    print("Training Complete")
    return model

In [5]:
def evaluate_model_on_test_set(model, test_loader):
    if not test_loader:
        return 0  # Handle None case
    
    model.eval()
    device = set_device()
    correct, total = 0, 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    
    acc = 100 * correct / total
    print(f" - Test Accuracy: {acc:.2f}%")
    return acc

In [6]:
def save_checkpoint(model, epoch, optimizer, best_acc, filename='./checkpoint/model_best_checkpoint.pth.tar'):
    os.makedirs("./checkpoint", exist_ok=True)  # Ensure directory exists
    state = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'best_accuracy': best_acc,
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(state, filename)
    print(f"Checkpoint saved at {filename}")  # Debugging print


In [7]:
def classify(model, image_transform, image_path, classes):
    model.eval()
    device = set_device()
    model.to(device)
    
    image = Image.open(image_path)
    image = image_transform(image).float().unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)
    
    print(f"Predicted Class: {classes[predicted.item()]}")
    return classes[predicted.item()]

In [8]:
def add_new_class(model, train_dataset_path, new_class_path, new_class_name, train_transforms, test_loader, n_epochs=10):
    class_dir = os.path.join(train_dataset_path, new_class_name)
    os.makedirs(class_dir, exist_ok=True)
    
    for img_file in os.listdir(new_class_path):
        shutil.copy(os.path.join(new_class_path, img_file), os.path.join(class_dir, img_file))
    
    train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    mean, std = get_mean_and_std(train_loader)
    print(f"Updated Mean: {mean}")
    print(f"Updated Std: {std}")
    
    train_transforms.transforms[-1] = transforms.Normalize(mean, std)
    train_dataset.transform = train_transforms
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))
    model.to(set_device())
    
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.003)
    criterion = nn.CrossEntropyLoss()
    
    model = train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs)
    return model, train_loader

In [9]:
train_dataset_path = "./Dataset"
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [10]:
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

In [11]:
mean, std = get_mean_and_std(train_loader)
print(f"Mean: {mean}, Std: {std}")
train_transforms.transforms.append(transforms.Normalize(mean, std))
train_dataset.transform = train_transforms
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

Mean: tensor([0.6700, 0.6752, 0.6902]), Std: tensor([0.1922, 0.1926, 0.2029])


In [12]:
resnet18_model = models.resnet18(pretrained=True)
resnet18_model.fc = nn.Linear(resnet18_model.fc.in_features, len(train_dataset.classes))
resnet18_model.to(set_device())

c:\Users\Omkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Omkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18_model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.003)
trained_model = train_nn(resnet18_model, train_loader, None, loss_fn, optimizer, 10)

Epoch 1
 - Training Accuracy: 52.44%. Loss: 0.6973
Epoch 2
 - Training Accuracy: 69.51%. Loss: 0.5131
Epoch 3
 - Training Accuracy: 70.73%. Loss: 0.3861
Epoch 4
 - Training Accuracy: 100.00%. Loss: 0.1950
Epoch 5
 - Training Accuracy: 98.78%. Loss: 0.1204
Epoch 6
 - Training Accuracy: 97.56%. Loss: 0.1357
Epoch 7
 - Training Accuracy: 100.00%. Loss: 0.0656
Epoch 8
 - Training Accuracy: 100.00%. Loss: 0.0373
Epoch 9
 - Training Accuracy: 100.00%. Loss: 0.0220
Epoch 10
 - Training Accuracy: 100.00%. Loss: 0.0280
Training Complete


In [14]:
if os.path.exists("./checkpoint/model_best_checkpoint.pth.tar"):
    checkpoint = torch.load("./checkpoint/model_best_checkpoint.pth.tar")
    resnet18_model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Best Accuracy: {checkpoint['best_accuracy']}, Epoch: {checkpoint['epoch']}")


In [15]:
torch.save(resnet18_model, "./models/best_model.pth")

In [16]:
classes = train_dataset.classes
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean.tolist(), std.tolist())
])

In [17]:
classify(torch.load('./models/best_model.pth'), image_transform, "./Dataset/Aadhar_card/my small aadhar.jpg", classes)

Predicted Class: Aadhar_card


C:\Users\Omkar\AppData\Local\Temp\ipykernel_5276\1098543544.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classify(torch.load('./models/best_model.pth'), image_transfo

'Aadhar_card'